In [ ]:
import huggingface_hub
huggingface_hub.notebook_login(new_session=False)

In [ ]:
import diffusers
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE in ("cuda", "mps") else torch.float32
print(f"Using {DEVICE}")

def get_generator(seed):
    if DEVICE in ("cuda", "mps"):
        return torch.Generator(DEVICE).manual_seed(seed)
    generator = torch.Generator()
    generator.manual_seed(seed)
    return generator

pipe = diffusers.StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    torch_dtype=DTYPE)

if DEVICE == "cuda":
    pipe = pipe.to(DEVICE)
    pipe.enable_model_cpu_offload()
elif DEVICE == "mps":
    pipe = pipe.to(DEVICE)
    pipe.enable_attention_slicing()

In [ ]:
import time

import diffusers
  
def gen_image(prompt, seed=1):
    start = time.time()
    img = pipe(
        prompt,
        negative_prompt="",
        num_inference_steps=28,
        guidance_scale=7.0,
        #height=512,
        #width=512,
        #height=1280,
        #width=720,
        generator=get_generator(seed),
    ).images[0]
    print(f"Generation took {time.time()-start:.1f}s")
    return img

In [ ]:
gen_image("A cat holding a sign that says hello world")

In [ ]:
gen_image("Busy street. Lovely couple. Flowers. After rain. Night life.")